# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="./figures/RAG-process.png" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [1]:
# #langchain library
# !pip install langchain==0.0.350
# #LLM
# !pip install accelerate==0.25.0
# !pip install transformers==4.36.2
# !pip install bitsandbytes==0.41.2
# #Text Embedding
# !pip install sentence-transformers==2.2.2
# !pip install InstructorEmbedding==1.0.1
# #vectorstore
# !pip install pymupdf==1.23.8
# !pip install faiss-gpu==1.7.2
# !pip install faiss-cpu==1.7.4

In [2]:
import os
import torch
# # Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [3]:
from langchain import PromptTemplate

prompt_template = """
    I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format 
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:")

In [4]:
PROMPT.format(
    context = "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can effectively generalize and thus perform tasks without explicit instructions.",
    question = "What is Machine Learning"
)

"I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can effectively generalize and thus perform tasks without explicit instructions.\n    Question: What is Machine Learning\n    Answer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code). 
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders 
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [5]:
from langchain.document_loaders import PyMuPDFLoader

nlp_docs = './docs/AIT-BROCHURE.pdf'

loader = PyMuPDFLoader(nlp_docs)
documents = loader.load()

In [6]:
# documents

In [7]:
len(documents)

2

In [8]:
documents[1]

Document(page_content='Department of Food, \nAgriculture and Bioresources\nAgribusiness Management \nAgricultural Systems and \nEngineering \nAquaculture and Aquatic \nResources Management \nFood Engineering and \nBioprocess Technology \nFood Innovation, Nutrition and \nHealth\nS T U DY  O P T I O N S  A T  A I T\nMaster’s and PhD Degree Programs\nAvailable in many fields of specializations across its \nthree Schools, covering various aspects of techno-\nlogy, social developments, and management. Sev-\neral new degree programs are designed to address \nemerging global challenges and to respond to cur-\nrent demands of the workforce. \nFlexible Degree Options\nNew alternative to traditional “On-campus Degree \nOptions”. Students can do many courses online via \nAIT’s Hybrid Modes of Instruction, with an option of \nflexible payment: course-by-course. The minimum \nenrollment is two courses per semester – ideal for those \nnot wanting to take too much time away from career, \nor those wa

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [10]:
doc[1]

Document(page_content='Regional Resource Center for Asia \nand Pacific \n--Advancement of Region’s environment \nand sustainable development goals \nthrough research and capacity building\nBelt & Road Research Center \n--Research and studies in sustainability \nissues of the Belt and Road region\nInternet Education and Research \nLaboratory \n--Development, training, and education \nprograms related to internet development \nand IT topics\nAIT Artificial Intelligence\nTechnology Center\n- - R e s e a r c h  a n d  d eve l o p m e n t  t o \nincorporate AI into AIT programs and its \napplication to real-world problems \nYunus Center at AIT\n- - E nte r p r i s e  s o l u t i o n s fo r  S D G s &', metadata={'source': './docs/AIT-BROCHURE.pdf', 'file_path': './docs/AIT-BROCHURE.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'producer': 'macOS Version 11.6 (Build 20G165) Quartz PDF

In [11]:
len(doc)

25

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [12]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [43]:
#locate vectorstore
vector_path = '../vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

create path done


In [14]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [15]:
#calling vector from local
vector_path = '../vector-store'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)   

In [16]:
#ready to use
retriever = vectordb.as_retriever()

In [17]:
retriever.get_relevant_documents("What is Dependency Parsing")

[Document(page_content='Department of Food, \nAgriculture and Bioresources\nAgribusiness Management \nAgricultural Systems and \nEngineering \nAquaculture and Aquatic \nResources Management \nFood Engineering and \nBioprocess Technology \nFood Innovation, Nutrition and \nHealth\nS T U DY  O P T I O N S  A T  A I T\nMaster’s and PhD Degree Programs\nAvailable in many fields of specializations across its \nthree Schools, covering various aspects of techno-\nlogy, social developments, and management. Sev-\neral new degree programs are designed to address \nemerging global challenges and to respond to cur-\nrent demands of the workforce. \nFlexible Degree Options\nNew alternative to traditional “On-campus Degree', metadata={'source': './docs/AIT-BROCHURE.pdf', 'file_path': './docs/AIT-BROCHURE.pdf', 'page': 1, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'producer': 'macOS Version 11.6 (Build 20G16

In [18]:
retriever.get_relevant_documents("What is Transformers")

[Document(page_content='Regenerative Sanitation \nSustainable Energy Transition\nDepartment of Development  \nand Sustainability\nDevelopment and Sustainability \nGender and Development \nStudies \nNatural Resources Management \nDevelopment Planning \nManagement and Innovation \nUrban Innovation and \nSustainability \nUrban Sustainability Planning \nand Design \nSociety and Environmental \nGovernance \nDepartment of Civil and \nInfrastructure Engineering\nConstruction Engineering and \nInfrastructure Management \nGeotechnical and Earth \nResources Engineering \nStructural Engineering \nTransportation Engineering \nWater Engineering and \nManagement \nDepartment of Information \nand Communication \nTechnologies \nComputer Science', metadata={'source': './docs/AIT-BROCHURE.pdf', 'file_path': './docs/AIT-BROCHURE.pdf', 'page': 1, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'producer': 'macOS Vers

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [19]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [20]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [21]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios. 
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [22]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [23]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time. 
- it only uses the last K interactions. 
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [24]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available), 
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [25]:
# #locate models
# model_path = './models'
# if not os.path.exists(model_path):
#     os.makedirs(model_path)
#     print('create path done')

In [26]:
# %cd ./models
# !git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

In [27]:
%cd ./models_gpt2
!git clone https://huggingface.co/anas-awadalla/gpt2-span-head-few-shot-k-16-finetuned-squad-seed-0

/Users/kyithinnu/GitHub/NLP-A7/models_gpt2
fatal: destination path 'gpt2-span-head-few-shot-k-16-finetuned-squad-seed-0' already exists and is not an empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

# model_id = '../models/fastchat-t5-3b-v1.0/'
# model_id = './models_gpt2/gpt2-span-head-few-shot-k-16-finetuned-squad-seed-0/'
model_id = '/Users/kyithinnu/GitHub/NLP-A7/models_gpt2/gpt2-span-head-few-shot-k-16-finetuned-squad-seed-0'

tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

# bitsandbyte_config = BitsAndBytesConfig(
#     load_in_4bit = True,
#     bnb_4bit_quant_type = "nf4",
#     bnb_4bit_compute_dtype = torch.float16,
#     bnb_4bit_use_double_quant = True
# )

# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config = bitsandbyte_config, #caution Nvidia
#     device_map = 'auto',
#     load_in_8bit = True
# )

model = AutoModelForCausalLM.from_pretrained(model_id)

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 10,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

In [29]:
# from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
# from transformers import BitsAndBytesConfig
# from langchain import HuggingFacePipeline
# import torch

# # model_id = '../models/fastchat-t5-3b-v1.0/'
# model_id = './models_gpt2/gpt2-span-head-few-shot-k-16-finetuned-squad-seed-0'

# tokenizer = AutoTokenizer.from_pretrained(
#     model_id)

# tokenizer.pad_token_id = tokenizer.eos_token_id

# bitsandbyte_config = BitsAndBytesConfig(
#     load_in_4bit = True,
#     bnb_4bit_quant_type = "nf4",
#     bnb_4bit_compute_dtype = torch.float16,
#     bnb_4bit_use_double_quant = True
# )

# model = AutoModelForSeq2SeqLM.from_pretrained(
#     model_id,
#     quantization_config = bitsandbyte_config, #caution Nvidia
#     device_map = 'auto',
#     load_in_8bit = True
# )

# pipe = pipeline(
#     task="text2text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens = 100,
#     model_kwargs = {
#         "temperature" : 0,
#         "repetition_penalty": 1.5
#     }
# )

# llm = HuggingFacePipeline(pipeline = pipe)

### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [30]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [31]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [32]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [33]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': '\n\nQA:\n\nA:\n'}

`combine_docs_chain`

In [34]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x2a447b250>)), document_variable_name='context')

In [35]:
query = "What is Transformers?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    Regenerative Sanitation 
Sustainable Energy Transition
Department of Development  
and Sustainability
Development and Sustainability 
Gender and Development 
Studies 
Natural Resources Management 
Development Planning 
Management and Innovation 
Urban Innovation and 
Sustainability 
Urban Sustainability Planning 
and Design 
Society and Environmental

{'input_documents': [Document(page_content='Regenerative Sanitation \nSustainable Energy Transition\nDepartment of Development  \nand Sustainability\nDevelopment and Sustainability \nGender and Development \nStudies \nNatural Resources Management \nDevelopment Planning \nManagement and Innovation \nUrban Innovation and \nSustainability \nUrban Sustainability Planning \nand Design \nSociety and Environmental \nGovernance \nDepartment of Civil and \nInfrastructure Engineering\nConstruction Engineering and \nInfrastructure Management \nGeotechnical and Earth \nResources Engineering \nStructural Engineering \nTransportation Engineering \nWater Engineering and \nManagement \nDepartment of Information \nand Communication \nTechnologies \nComputer Science', metadata={'source': './docs/AIT-BROCHURE.pdf', 'file_path': './docs/AIT-BROCHURE.pdf', 'page': 1, 'total_pages': 2, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'pr

In [36]:
memory = ConversationBufferWindowMemory(
    k=3, 
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerati

## 5. Chatbot

In [37]:
prompt_question = "What is AIT?"
answer = chain({"question":prompt_question})
answer

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    • United Nations Economic and Social  
Commission for Asia and the Pacific
• United States Agency for International Development 
• World Bank
• and more …
P A R T N E R S  
A B O U T
A I T
The Asian Institute of Technology (AIT) is an 
international English-speaking postgraduate 
institution, pro

{'question': 'What is AIT?',
 'chat_history': [],
 'answer': ' AIT is an interdisciplinary international institute of applied',
 'source_documents': [Document(page_content='• United Nations Economic and Social  \nCommission for Asia and the Pacific\n• United States Agency for International Development \n• World Bank\n• and more …\nP A R T N E R S  \nA B O U T\nA I T\nThe Asian Institute of Technology (AIT) is an \ninternational English-speaking postgraduate \ninstitution, providing engineering, environment, \nand management education and research, \nfocusing on Sustainability and Social Impact \nissues. Our current strategic themed areas \nare: Climate Change; Smart Communities; \nFood- Energy-Water; Infrastr ucture; and \nTechnology, Policy and Society. AIT’s rigorous \nacademic, research, and outreach programs \nprepare graduates for professional success and', metadata={'source': './docs/AIT-BROCHURE.pdf', 'file_path': './docs/AIT-BROCHURE.pdf', 'page': 1, 'total_pages': 2, 'format':

In [38]:
prompt_question = "What is scholarship?"
answer = chain({"question":prompt_question})
answer

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='What is AIT?'), AIMessage(content=' AIT is an interdisciplinary international institute of applied')]
Follow Up Input: What is scholarship?
Standalone question:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    STRONG FOCUS
on research and interactive studies in 
sustainability areas
HANDS-ON
experiential learning and real-world 
application through exchanges, 
internships, service learning, and 
entrepreneurship
PERSONAL
attention by faculty and staff to students 
who are the institute’s highest priority
ACCESS
to experienced business ex

{'question': 'What is scholarship?',
 'chat_history': [HumanMessage(content='What is AIT?'),
  AIMessage(content=' AIT is an interdisciplinary international institute of applied')],
 'answer': '  Are scholars interested in this question? \n',
 'source_documents': [Document(page_content='STRONG FOCUS\non research and interactive studies in \nsustainability areas\nHANDS-ON\nexperiential learning and real-world \napplication through exchanges, \ninternships, service learning, and \nentrepreneurship\nPERSONAL\nattention by faculty and staff to students \nwho are the institute’s highest priority\nACCESS\nto experienced business executives, \nenterprises, NGOs and governments  \nin Asia\nNETWORK\nof internationally recognized  \npartner universities\nCONNECTION\nto over 25,000 postgraduate level alumni \nfrom 100+ countries\nM s .  X i a o m i n g  Ta o \n(Class of 2014)\nCurrent Position \nProject Manager, DFMT Consulting Co, China\nStudying at AIT was a unique and enjoy-', metadata={'sourc

In [39]:
prompt_question = "Where is AIT?"
answer = chain({"question":prompt_question})
answer

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='What is AIT?'), AIMessage(content=' AIT is an interdisciplinary international institute of applied'), HumanMessage(content='What is scholarship?'), AIMessage(content='  Are scholars interested in this question? \n')]
Follow Up Input: Where is AIT?
Standalone question:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    process that has taken me thus far.
D r.  B i n d a  P a n d e y 
(Class of 2002)
Current Position 
MP, House of Representative and Chair of Trade Union 
Policy Institute of GEFONT, Nepal
AIT has added me theoretical clarity which 
has helped me in my role as a woman and 
a trade union activist dealing with gender 
issues for decad

{'question': 'Where is AIT?',
 'chat_history': [HumanMessage(content='What is AIT?'),
  AIMessage(content=' AIT is an interdisciplinary international institute of applied'),
  HumanMessage(content='What is scholarship?'),
  AIMessage(content='  Are scholars interested in this question? \n')],
 'answer': ' __________________ This paper was co-cited',
 'source_documents': [Document(page_content='process that has taken me thus far.\nD r.  B i n d a  P a n d e y \n(Class of 2002)\nCurrent Position \nMP, House of Representative and Chair of Trade Union \nPolicy Institute of GEFONT, Nepal\nAIT has added me theoretical clarity which \nhas helped me in my role as a woman and \na trade union activist dealing with gender \nissues for decades. I could apply my aca-\ndemic knowledge to my work on move-\nment and state policy to achieve gender \njustice. I am always thankful to AIT for \nenriching me with this academic strength.\nM r.  S o p h e a p  S r e n g \n(Class of 2010) \nCurrent Position \